# 1NN vs RandomForest

В этом задании будет использоваться датасет digits из sklearn.datasets. Оставьте последние 25% объектов для контроля качества, разделив X и y на X_train, y_train и X_test, y_test.

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами 1NN с RandomForestClassifier из sklearn на 1000 деревьях.



## Задание 1

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка N log N сравнений (строже говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в KNeighborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных ball tree и kd tree.

Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1.

In [81]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

digits = load_digits()
X, y = digits.data, digits.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, shuffle=False)

In [82]:
import numpy as np
from numpy import linalg

class OneNearestNeighbor(object):
    

    def __init__(self):
        pass

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
          
    def dist(self, x, y): 
        return linalg.norm(x - y)
    
    def predict(self, X):
        """slow version with two loops"""
        ans = list()
        for x_test in X:
            cur_dist = list()
            for x_train in self.X_train:
                cur_dist.append(self.dist(x_test, x_train))
            min_ind = np.argmin(cur_dist)
            ans.append(y_train[min_ind])
        return np.array(ans)

In [83]:
def accuracy(y_pred, y_test):
    return np.sum(np.array(y_pred) != np.array(y_test))/len(y_test)

In [84]:
clf = OneNearestNeighbor()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
accuracy(y_pred, y_test)

0.03777777777777778

## Задание 2

Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [85]:
from sklearn import ensemble, model_selection, metrics 

import numpy as np
import pandas as pd

rf_clf = ensemble.RandomForestClassifier(n_estimators = 1000)
rf_clf.fit(X_train, y_train)

y_pred_rf = rf_clf.predict(X_test)
accuracy(y_pred_rf, y_test)

0.06666666666666667

In [86]:
def write_ans(file_name, ans):
    with open(file_name, "w") as fout:
        fout.write(str(ans))
        
write_ans("1.txt", accuracy(y_pred, y_test))
write_ans("2.txt", accuracy(y_pred_rf, y_test))